In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import sem
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import linregress
from scipy.interpolate import *
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from statsmodels.api import add_constant

# File to Load (Remember to Change These)
MSPG = "MostStatsPerGame.csv"

# Read the Mouse and Drug Data and the Clinical Trial Data
MSPG_df = pd.read_csv(MSPG)
MSPG_df=MSPG_df.fillna(0)
MSPG_df


MSPG_df['PID'] = MSPG_df['Key'].str[:36]
MSPG_df

,Key,Player Name,Player Team,Season,Position,Games Played,Games Started,Season Age,Rec Red Zone Targets PG,Rec Receptions PG,...,Passing Redzone Attempts PG,Passing Touchdowns PG,Passing Yards PG,Field Goal Attempts PG,Field Goal Longest,Field Goal Made PG,Field Goal Yards PG,Extra Pts Attempts PG,Extra Pts Made PG,PID
0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2012 768c...,Ryan Fitzpatrick,Bills,2012,QB,16,16,30,0.000000,0.000000,...,0.312500,1.500000,212.500000,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
1,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2013 d26a...,Ryan Fitzpatrick,Titans,2013,QB,11,9,31,0.090909,0.090909,...,3.727273,1.272727,223.090909,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
2,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2014 82d2...,Ryan Fitzpatrick,Texans,2014,QB,12,12,32,0.000000,0.000000,...,3.166667,1.416667,206.916667,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
3,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2015 5fee...,Ryan Fitzpatrick,Jets,2015,QB,16,16,33,0.000000,0.000000,...,4.500000,1.937500,244.062500,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
4,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2016 5fee...,Ryan Fitzpatrick,Jets,2016,QB,14,11,34,0.000000,0.000000,...,1.928571,0.857143,193.571429,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
5,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2017 4254...,Ryan Fitzpatrick,Buccaneers,2017,QB,6,3,35,0.000000,0.000000,...,1.666667,1.166667,183.833333,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
6,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2018 4254...,Ryan Fitzpatrick,Buccaneers,2018,QB,8,7,36,0.000000,0.000000,...,2.625000,2.125000,295.750000,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
7,8fc65820-f565-44e2-8635-3e1cdf165bf6 2012 768c...,Chris Hogan,Bills,2012,WR,0,0,25,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6
8,8fc65820-f565-44e2-8635-3e1cdf165bf6 2013 768c...,Chris Hogan,Bills,2013,WR,16,0,26,0.000000,0.625000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6
9,8fc65820-f565-44e2-8635-3e1cdf165bf6 2014 768c...,Chris Hogan,Bills,2014,WR,16,2,27,0.562500,2.562500,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6


In [2]:

f2 = MSPG_df[["PID", "Season"]].drop_duplicates()
f3 = f2.groupby("PID")["Season"].count().reset_index(name="Season Count")
f4 = f3.groupby("Season Count")["PID"].count().reset_index(name="Player Count")
f4

,Season Count,Player Count
0,1,168
1,2,136
2,3,99
3,4,74
4,5,58
5,6,56
6,7,120


In [3]:
YR2018_df = f2[f2["Season"] == 2018]
player_2018 = YR2018_df["PID"].tolist()

YR2017_df = f2[f2["Season"] == 2017]
player_2017 = YR2017_df["PID"].tolist()

YR2016_df = f2[f2["Season"] == 2016]
player_2016 = YR2016_df["PID"].tolist()

YR2015_df = f2[f2["Season"] == 2015]
player_2015 = YR2015_df["PID"].tolist()

YR2014_df = f2[f2["Season"] == 2014]
player_2014 = YR2014_df["PID"].tolist()

YR2013_df = f2[f2["Season"] == 2013]
player_2013 = YR2013_df["PID"].tolist()

YR2012_df = f2[f2["Season"] == 2012]
player_2012 = YR2012_df["PID"].tolist()



In [4]:
#2018: rookies to exclude

test = MSPG_df


rookie2018_list = []

for i in range (len(player_2018)): 
    if player_2018[i] not in player_2017:
        if player_2018[i] not in player_2016:
            if player_2018[i] not in player_2015: 
                if player_2018[i] not in player_2014:
                    if player_2018[i] not in player_2013:
                        if player_2018[i] not in player_2012:
                            rookie2018_list.append(player_2018[i])
                        else:
                            continue
                    else:
                        continue
                else:
                    continue
            else:
                continue
        else:
            continue
    else:
        continue

#print(len(player_2018))
#print(len(rookie2018_list))

#this is how you filter a dataframe from a list

test2 = test[~test['PID'].isin(rookie2018_list)]
test2
#test2 is dataset w/o 2018 rookies

,Key,Player Name,Player Team,Season,Position,Games Played,Games Started,Season Age,Rec Red Zone Targets PG,Rec Receptions PG,...,Passing Redzone Attempts PG,Passing Touchdowns PG,Passing Yards PG,Field Goal Attempts PG,Field Goal Longest,Field Goal Made PG,Field Goal Yards PG,Extra Pts Attempts PG,Extra Pts Made PG,PID
0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2012 768c...,Ryan Fitzpatrick,Bills,2012,QB,16,16,30,0.000000,0.000000,...,0.312500,1.500000,212.500000,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
1,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2013 d26a...,Ryan Fitzpatrick,Titans,2013,QB,11,9,31,0.090909,0.090909,...,3.727273,1.272727,223.090909,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
2,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2014 82d2...,Ryan Fitzpatrick,Texans,2014,QB,12,12,32,0.000000,0.000000,...,3.166667,1.416667,206.916667,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
3,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2015 5fee...,Ryan Fitzpatrick,Jets,2015,QB,16,16,33,0.000000,0.000000,...,4.500000,1.937500,244.062500,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
4,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2016 5fee...,Ryan Fitzpatrick,Jets,2016,QB,14,11,34,0.000000,0.000000,...,1.928571,0.857143,193.571429,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
5,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2017 4254...,Ryan Fitzpatrick,Buccaneers,2017,QB,6,3,35,0.000000,0.000000,...,1.666667,1.166667,183.833333,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
6,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2018 4254...,Ryan Fitzpatrick,Buccaneers,2018,QB,8,7,36,0.000000,0.000000,...,2.625000,2.125000,295.750000,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
7,8fc65820-f565-44e2-8635-3e1cdf165bf6 2012 768c...,Chris Hogan,Bills,2012,WR,0,0,25,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6
8,8fc65820-f565-44e2-8635-3e1cdf165bf6 2013 768c...,Chris Hogan,Bills,2013,WR,16,0,26,0.000000,0.625000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6
9,8fc65820-f565-44e2-8635-3e1cdf165bf6 2014 768c...,Chris Hogan,Bills,2014,WR,16,2,27,0.562500,2.562500,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6


In [5]:
#figure out spread

#2017-2018

go_df = test2

no_rookie18 = []
no_rookie18_un = []
p17_18 = []

no_rookie18 = go_df["PID"].tolist()


for x in no_rookie18: 
    if x not in no_rookie18_un: 
        no_rookie18_un.append(x) 

for i in range (len(no_rookie18_un)): 
    if no_rookie18_un[i] in player_2017:
        p17_18.append(no_rookie18_un[i])
    else:
        continue
                    

#print(len(player_2018))
#print(len(rookie2018_list))

#this is how you filter a dataframe from a list

test3 = go_df[go_df['PID'].isin(p17_18)]
test3
#test3 is dataset w 2018 & 2017 players


,Key,Player Name,Player Team,Season,Position,Games Played,Games Started,Season Age,Rec Red Zone Targets PG,Rec Receptions PG,...,Passing Redzone Attempts PG,Passing Touchdowns PG,Passing Yards PG,Field Goal Attempts PG,Field Goal Longest,Field Goal Made PG,Field Goal Yards PG,Extra Pts Attempts PG,Extra Pts Made PG,PID
0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2012 768c...,Ryan Fitzpatrick,Bills,2012,QB,16,16,30,0.000000,0.000000,...,0.312500,1.500000,212.500000,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
1,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2013 d26a...,Ryan Fitzpatrick,Titans,2013,QB,11,9,31,0.090909,0.090909,...,3.727273,1.272727,223.090909,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
2,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2014 82d2...,Ryan Fitzpatrick,Texans,2014,QB,12,12,32,0.000000,0.000000,...,3.166667,1.416667,206.916667,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
3,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2015 5fee...,Ryan Fitzpatrick,Jets,2015,QB,16,16,33,0.000000,0.000000,...,4.500000,1.937500,244.062500,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
4,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2016 5fee...,Ryan Fitzpatrick,Jets,2016,QB,14,11,34,0.000000,0.000000,...,1.928571,0.857143,193.571429,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
5,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2017 4254...,Ryan Fitzpatrick,Buccaneers,2017,QB,6,3,35,0.000000,0.000000,...,1.666667,1.166667,183.833333,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
6,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2018 4254...,Ryan Fitzpatrick,Buccaneers,2018,QB,8,7,36,0.000000,0.000000,...,2.625000,2.125000,295.750000,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
7,8fc65820-f565-44e2-8635-3e1cdf165bf6 2012 768c...,Chris Hogan,Bills,2012,WR,0,0,25,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6
8,8fc65820-f565-44e2-8635-3e1cdf165bf6 2013 768c...,Chris Hogan,Bills,2013,WR,16,0,26,0.000000,0.625000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6
9,8fc65820-f565-44e2-8635-3e1cdf165bf6 2014 768c...,Chris Hogan,Bills,2014,WR,16,2,27,0.562500,2.562500,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6


In [6]:
#figure out spread

#2016-2018

go_df = test3

p17_18 = []
p17_18_un = []
p16_17_18 = []

p17_18 = go_df["PID"].tolist()


for x in p17_18: 
    if x not in p17_18_un: 
        p17_18_un.append(x) 

for i in range (len(p17_18_un)): 
    if p17_18_un[i] in player_2016:
        p16_17_18.append(p17_18_un[i])
    else:
        continue
                    

#print(len(player_2018))
#print(len(rookie2018_list))

#this is how you filter a dataframe from a list

test4 = go_df[go_df['PID'].isin(p16_17_18)]
test4
#test4 is dataset w 2018 & 2017 & 2016 players

,Key,Player Name,Player Team,Season,Position,Games Played,Games Started,Season Age,Rec Red Zone Targets PG,Rec Receptions PG,...,Passing Redzone Attempts PG,Passing Touchdowns PG,Passing Yards PG,Field Goal Attempts PG,Field Goal Longest,Field Goal Made PG,Field Goal Yards PG,Extra Pts Attempts PG,Extra Pts Made PG,PID
0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2012 768c...,Ryan Fitzpatrick,Bills,2012,QB,16,16,30,0.000000,0.000000,...,0.312500,1.500000,212.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
1,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2013 d26a...,Ryan Fitzpatrick,Titans,2013,QB,11,9,31,0.090909,0.090909,...,3.727273,1.272727,223.090909,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
2,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2014 82d2...,Ryan Fitzpatrick,Texans,2014,QB,12,12,32,0.000000,0.000000,...,3.166667,1.416667,206.916667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
3,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2015 5fee...,Ryan Fitzpatrick,Jets,2015,QB,16,16,33,0.000000,0.000000,...,4.500000,1.937500,244.062500,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
4,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2016 5fee...,Ryan Fitzpatrick,Jets,2016,QB,14,11,34,0.000000,0.000000,...,1.928571,0.857143,193.571429,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
5,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2017 4254...,Ryan Fitzpatrick,Buccaneers,2017,QB,6,3,35,0.000000,0.000000,...,1.666667,1.166667,183.833333,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
6,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2018 4254...,Ryan Fitzpatrick,Buccaneers,2018,QB,8,7,36,0.000000,0.000000,...,2.625000,2.125000,295.750000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
7,8fc65820-f565-44e2-8635-3e1cdf165bf6 2012 768c...,Chris Hogan,Bills,2012,WR,0,0,25,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,8fc65820-f565-44e2-8635-3e1cdf165bf6
8,8fc65820-f565-44e2-8635-3e1cdf165bf6 2013 768c...,Chris Hogan,Bills,2013,WR,16,0,26,0.000000,0.625000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,8fc65820-f565-44e2-8635-3e1cdf165bf6
9,8fc65820-f565-44e2-8635-3e1cdf165bf6 2014 768c...,Chris Hogan,Bills,2014,WR,16,2,27,0.562500,2.562500,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,8fc65820-f565-44e2-8635-3e1cdf165bf6


In [7]:
#figure out spread

#2015-2018

go_df = test4

p16_17_18 = []
p16_17_18_un = []
p15_16_17_18 = []

p16_17_18 = go_df["PID"].tolist()


for x in p16_17_18: 
    if x not in p16_17_18_un: 
        p16_17_18_un.append(x) 

for i in range (len(p16_17_18_un)): 
    if p16_17_18_un[i] in player_2015:
        p15_16_17_18.append(p16_17_18_un[i])
    else:
        continue
                    

#print(len(player_2018))
#print(len(rookie2018_list))

#this is how you filter a dataframe from a list

test5 = go_df[go_df['PID'].isin( p15_16_17_18)]
test5
#test5 is dataset w 2018 & 2017 & 2016 & 2015 players

,Key,Player Name,Player Team,Season,Position,Games Played,Games Started,Season Age,Rec Red Zone Targets PG,Rec Receptions PG,...,Passing Redzone Attempts PG,Passing Touchdowns PG,Passing Yards PG,Field Goal Attempts PG,Field Goal Longest,Field Goal Made PG,Field Goal Yards PG,Extra Pts Attempts PG,Extra Pts Made PG,PID
0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2012 768c...,Ryan Fitzpatrick,Bills,2012,QB,16,16,30,0.000000,0.000000,...,0.312500,1.500000,212.500000,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
1,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2013 d26a...,Ryan Fitzpatrick,Titans,2013,QB,11,9,31,0.090909,0.090909,...,3.727273,1.272727,223.090909,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
2,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2014 82d2...,Ryan Fitzpatrick,Texans,2014,QB,12,12,32,0.000000,0.000000,...,3.166667,1.416667,206.916667,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
3,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2015 5fee...,Ryan Fitzpatrick,Jets,2015,QB,16,16,33,0.000000,0.000000,...,4.500000,1.937500,244.062500,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
4,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2016 5fee...,Ryan Fitzpatrick,Jets,2016,QB,14,11,34,0.000000,0.000000,...,1.928571,0.857143,193.571429,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
5,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2017 4254...,Ryan Fitzpatrick,Buccaneers,2017,QB,6,3,35,0.000000,0.000000,...,1.666667,1.166667,183.833333,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
6,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2018 4254...,Ryan Fitzpatrick,Buccaneers,2018,QB,8,7,36,0.000000,0.000000,...,2.625000,2.125000,295.750000,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
7,8fc65820-f565-44e2-8635-3e1cdf165bf6 2012 768c...,Chris Hogan,Bills,2012,WR,0,0,25,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6
8,8fc65820-f565-44e2-8635-3e1cdf165bf6 2013 768c...,Chris Hogan,Bills,2013,WR,16,0,26,0.000000,0.625000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6
9,8fc65820-f565-44e2-8635-3e1cdf165bf6 2014 768c...,Chris Hogan,Bills,2014,WR,16,2,27,0.562500,2.562500,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6


In [8]:
#figure out spread

#2014-2018

go_df = test5

p15_16_17_18 = []
p15_16_17_18_un = []
p14_15_16_17_18 = []

p15_16_17_18 = go_df["PID"].tolist()


for x in p15_16_17_18: 
    if x not in p15_16_17_18_un: 
        p15_16_17_18_un.append(x) 

for i in range (len(p15_16_17_18_un)): 
    if p15_16_17_18_un[i] in player_2014:
        p14_15_16_17_18.append(p15_16_17_18_un[i])
    else:
        continue
                    

#print(len(player_2018))
#print(len(rookie2018_list))

#this is how you filter a dataframe from a list

test6 = go_df[go_df['PID'].isin(p14_15_16_17_18)]
test6
#test6 is dataset w 2018 & 2017 & 2016 & 2015 & 2014 players

,Key,Player Name,Player Team,Season,Position,Games Played,Games Started,Season Age,Rec Red Zone Targets PG,Rec Receptions PG,...,Passing Redzone Attempts PG,Passing Touchdowns PG,Passing Yards PG,Field Goal Attempts PG,Field Goal Longest,Field Goal Made PG,Field Goal Yards PG,Extra Pts Attempts PG,Extra Pts Made PG,PID
0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2012 768c...,Ryan Fitzpatrick,Bills,2012,QB,16,16,30,0.000000,0.000000,...,0.312500,1.500000,212.500000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
1,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2013 d26a...,Ryan Fitzpatrick,Titans,2013,QB,11,9,31,0.090909,0.090909,...,3.727273,1.272727,223.090909,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
2,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2014 82d2...,Ryan Fitzpatrick,Texans,2014,QB,12,12,32,0.000000,0.000000,...,3.166667,1.416667,206.916667,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
3,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2015 5fee...,Ryan Fitzpatrick,Jets,2015,QB,16,16,33,0.000000,0.000000,...,4.500000,1.937500,244.062500,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
4,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2016 5fee...,Ryan Fitzpatrick,Jets,2016,QB,14,11,34,0.000000,0.000000,...,1.928571,0.857143,193.571429,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
5,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2017 4254...,Ryan Fitzpatrick,Buccaneers,2017,QB,6,3,35,0.000000,0.000000,...,1.666667,1.166667,183.833333,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
6,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2018 4254...,Ryan Fitzpatrick,Buccaneers,2018,QB,8,7,36,0.000000,0.000000,...,2.625000,2.125000,295.750000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
7,8fc65820-f565-44e2-8635-3e1cdf165bf6 2012 768c...,Chris Hogan,Bills,2012,WR,0,0,25,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,8fc65820-f565-44e2-8635-3e1cdf165bf6
8,8fc65820-f565-44e2-8635-3e1cdf165bf6 2013 768c...,Chris Hogan,Bills,2013,WR,16,0,26,0.000000,0.625000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,8fc65820-f565-44e2-8635-3e1cdf165bf6
9,8fc65820-f565-44e2-8635-3e1cdf165bf6 2014 768c...,Chris Hogan,Bills,2014,WR,16,2,27,0.562500,2.562500,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,8fc65820-f565-44e2-8635-3e1cdf165bf6


In [9]:
#figure out spread

#2013-2018

go_df = test6

p14_15_16_17_18 = []
p14_15_16_17_18_un = []
p13_14_15_16_17_18 = []

p14_15_16_17_18 = go_df["PID"].tolist()


for x in p14_15_16_17_18: 
    if x not in p14_15_16_17_18_un: 
        p14_15_16_17_18_un.append(x) 

for i in range (len(p14_15_16_17_18_un)): 
    if p14_15_16_17_18_un[i] in player_2013:
        p13_14_15_16_17_18.append(p14_15_16_17_18_un[i])
    else:
        continue
                    

#print(len(player_2018))
#print(len(rookie2018_list))

#this is how you filter a dataframe from a list

test7 = go_df[go_df['PID'].isin(p13_14_15_16_17_18)]
test7
#test7 is dataset w 2018 & 2017 & 2016 & 2015 & 2014 & 2013 players

,Key,Player Name,Player Team,Season,Position,Games Played,Games Started,Season Age,Rec Red Zone Targets PG,Rec Receptions PG,...,Passing Redzone Attempts PG,Passing Touchdowns PG,Passing Yards PG,Field Goal Attempts PG,Field Goal Longest,Field Goal Made PG,Field Goal Yards PG,Extra Pts Attempts PG,Extra Pts Made PG,PID
0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2012 768c...,Ryan Fitzpatrick,Bills,2012,QB,16,16,30,0.000000,0.000000,...,0.312500,1.500000,212.500000,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
1,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2013 d26a...,Ryan Fitzpatrick,Titans,2013,QB,11,9,31,0.090909,0.090909,...,3.727273,1.272727,223.090909,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
2,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2014 82d2...,Ryan Fitzpatrick,Texans,2014,QB,12,12,32,0.000000,0.000000,...,3.166667,1.416667,206.916667,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
3,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2015 5fee...,Ryan Fitzpatrick,Jets,2015,QB,16,16,33,0.000000,0.000000,...,4.500000,1.937500,244.062500,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
4,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2016 5fee...,Ryan Fitzpatrick,Jets,2016,QB,14,11,34,0.000000,0.000000,...,1.928571,0.857143,193.571429,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
5,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2017 4254...,Ryan Fitzpatrick,Buccaneers,2017,QB,6,3,35,0.000000,0.000000,...,1.666667,1.166667,183.833333,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
6,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2018 4254...,Ryan Fitzpatrick,Buccaneers,2018,QB,8,7,36,0.000000,0.000000,...,2.625000,2.125000,295.750000,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
7,8fc65820-f565-44e2-8635-3e1cdf165bf6 2012 768c...,Chris Hogan,Bills,2012,WR,0,0,25,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6
8,8fc65820-f565-44e2-8635-3e1cdf165bf6 2013 768c...,Chris Hogan,Bills,2013,WR,16,0,26,0.000000,0.625000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6
9,8fc65820-f565-44e2-8635-3e1cdf165bf6 2014 768c...,Chris Hogan,Bills,2014,WR,16,2,27,0.562500,2.562500,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6


In [12]:
#figure out spread

#2012-2018

go_df = test7

p13_14_15_16_17_18  = []
p13_14_15_16_17_18_un = []
p12_13_14_15_16_17_18 = []

p13_14_15_16_17_18 = go_df["PID"].tolist()


for x in p13_14_15_16_17_18: 
    if x not in p13_14_15_16_17_18_un: 
        p13_14_15_16_17_18_un.append(x) 

for i in range (len(p13_14_15_16_17_18_un)): 
    if p13_14_15_16_17_18_un[i] in player_2012:
        p12_13_14_15_16_17_18.append(p13_14_15_16_17_18_un[i])
    else:
        continue
                    

#print(len(player_2018))
#print(len(rookie2018_list))

#this is how you filter a dataframe from a list

test8 = go_df[go_df['PID'].isin(p12_13_14_15_16_17_18)]
test8
#test8 is dataset w 2018 & 2017 & 2016 & 2015 & 2014 & 2013 & 2012 players

,Key,Player Name,Player Team,Season,Position,Games Played,Games Started,Season Age,Rec Red Zone Targets PG,Rec Receptions PG,...,Passing Redzone Attempts PG,Passing Touchdowns PG,Passing Yards PG,Field Goal Attempts PG,Field Goal Longest,Field Goal Made PG,Field Goal Yards PG,Extra Pts Attempts PG,Extra Pts Made PG,PID
0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2012 768c...,Ryan Fitzpatrick,Bills,2012,QB,16,16,30,0.000000,0.000000,...,0.312500,1.500000,212.500000,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
1,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2013 d26a...,Ryan Fitzpatrick,Titans,2013,QB,11,9,31,0.090909,0.090909,...,3.727273,1.272727,223.090909,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
2,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2014 82d2...,Ryan Fitzpatrick,Texans,2014,QB,12,12,32,0.000000,0.000000,...,3.166667,1.416667,206.916667,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
3,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2015 5fee...,Ryan Fitzpatrick,Jets,2015,QB,16,16,33,0.000000,0.000000,...,4.500000,1.937500,244.062500,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
4,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2016 5fee...,Ryan Fitzpatrick,Jets,2016,QB,14,11,34,0.000000,0.000000,...,1.928571,0.857143,193.571429,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
5,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2017 4254...,Ryan Fitzpatrick,Buccaneers,2017,QB,6,3,35,0.000000,0.000000,...,1.666667,1.166667,183.833333,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
6,0742d2ea-1cf2-49a6-a150-77ba6e034d8c 2018 4254...,Ryan Fitzpatrick,Buccaneers,2018,QB,8,7,36,0.000000,0.000000,...,2.625000,2.125000,295.750000,0.0,0.0,0.0,0.0,0.0,0.0,0742d2ea-1cf2-49a6-a150-77ba6e034d8c
7,8fc65820-f565-44e2-8635-3e1cdf165bf6 2012 768c...,Chris Hogan,Bills,2012,WR,0,0,25,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6
8,8fc65820-f565-44e2-8635-3e1cdf165bf6 2013 768c...,Chris Hogan,Bills,2013,WR,16,0,26,0.000000,0.625000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6
9,8fc65820-f565-44e2-8635-3e1cdf165bf6 2014 768c...,Chris Hogan,Bills,2014,WR,16,2,27,0.562500,2.562500,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8fc65820-f565-44e2-8635-3e1cdf165bf6


In [11]:
test3
t1 = test3[["PID", "Season"]].drop_duplicates()
t2 = t1.groupby("PID")["Season"].count().reset_index(name="Season Count")
t3 = t2.groupby("Season Count")["PID"].count().reset_index(name="Player Count")
t3

#test4
#t4 = test4[["PID", "Season"]].drop_duplicates()
#t5 = t4.groupby("PID")["Season"].count().reset_index(name="Season Count")
#t6 = t5.groupby("Season Count")["PID"].count().reset_index(name="Player Count")
#t6

#test5
#test6
#test7
#test8

,Season Count,Player Count
0,2,121
1,3,91
2,4,70
3,5,57
4,6,51
5,7,120
